In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import time
import os
import spacy
import urllib.parse as url
import re, string, unicodedata
from psaw import PushshiftAPI

In [2]:
nlp=spacy.load('en_core_web_sm')

In [3]:
os.chdir('../')

In [32]:
api = PushshiftAPI()
#subred_list = ['nyc','LosAngeles','houston', 'chicago','philadelphia']
#keyword_list = ['asians', 'Asian%20American', 'asian%20people','asian%20person', 'zzxibussw']
start_epoch=int(dt.datetime(2018, 7, 2).timestamp())
#end_epoch = int(dt.datetime(2018, 2, 1).timestamp())
end_epoch = int(dt.datetime(2018, 12, 31).timestamp())

In [33]:
#keyword_list = ['asian%20american', 'Asian%20people','Asian%20person']
keyword_list = ['asians','blacks']


In [8]:
def convert_ascii_to_string(text):
    """convert ascii text to string"""
    import html
    return html.unescape(text)

In [9]:
def remove_URL(text):
    """Remove URLs from a sample string"""
    return re.sub(r"http\S+", "", text)

In [10]:
def replace_period(text):
    """Remove URLs from a sample string"""
    return re.sub(r"\w$", "\g<0>.", text)

In [11]:
def remove_punct(word):
    """Remove punctuation from sample string"""
    list_of_char = ['>', '*', '\"', '#', ]
    pattern = '[' + ''.join(list_of_char) + ']'
    return re.sub(pattern, '', word)

In [12]:
def remove_whitespace(text):
    """remove extra whitespaces from text"""
    text = text.strip()
    return " ".join(text.split())

In [13]:
def capitalize(text):
    """captilaize first word in sentence"""
    return text.capitalize()

In [14]:
def remove_leadspace(text):
    """remove leading whitespaces from text"""
    text = text.strip()
    return " ".join(text.split())
    #return re.sub(r'\t+', "", text)

In [15]:
def convert_list_to_string(text, seperator=' '):
    """ Convert list to string, by joining all item in list with given separator.
        Returns the concatenated string """
    return seperator.join(text)

In [16]:
def remove_non_ascii(text):
    return unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')

In [17]:
def remove_multi_periods(text):
    """remove ellipses from text and replace with a single period"""
    return re.sub(r'\.+', ". ", text)

In [18]:
def remove_multi_xmarks(text):
    """remove repeating exlamations from text and replace with a single one"""
    return re.sub(r'\!+', "! ", text)

In [19]:
def remove_multi_qmarks(text):
    """remove repeating question marks from text and replace with a single one"""
    return re.sub(r'\?+', "? ", text)

In [20]:
def get_sentences(text,k):
    doc = nlp(str(text))
    result = ''
    for sent in doc.sents:
        if url.unquote(k).casefold() in str(sent.text).casefold():
            if len(str(sent)) < 25:
                continue
            if "bot".casefold() in str(sent.text).casefold():
                continue
            if '[' in str(sent.text) or '(' in str(sent.text):
                continue
            result += capitalize(replace_period(str(sent.text))) + '\n'
    return result

In [34]:
def getData(keyword):
    results = api.search_comments( q=keyword, after=start_epoch, before=end_epoch)#,limit = 2000)
    cache = []
    ID = []
    for c in results:
        cache.append(c)
    return cache

In [37]:
replace_words = {'asians':'Asians','america':'America','whites':'Whites','arab':'Arab',
                'europe':'Europe','hispanic':'Hispanic','latin':'Latin','africa':'Africa',
                'mexic':'Mexic','blacks':'Blacks','india':'India','filipin':'Filipin',
                'korea':'Korea','china':'China','japan':'Japan','chinese':'Chinese',
                'caucasian':'Caucasian','asian American':'Asian American','north America':'North America',
                'middle east':'Middle East','white America':'White America','black America':'Black America',
                'white pe':'White pe','black pe':'Black pe','pakistan':'Pakistan',
                'black wom':'Black wom','white wom':'White wom','black man':'Black man',
                'white man':'White man','black men':'Black men','white men':'White men',
                'native American':'Native American','brazil':'Brazil','asian man':'Asian man',
                'asian wom':'Asian wom','asian pe':'Asian pe',}                

In [38]:
def saveResults(sub_list,keyword):
    commentData = []
    sentence = []
    for comment in sub_list:
        commentData.append([comment.body])
    df = pd.DataFrame(commentData,columns=['comments'])
    df.style.set_properties(subset=["comments"], **{'text-align': 'left'})
    df.comments = df.comments.apply(convert_ascii_to_string).astype('U').values
    df.comments = df.comments.apply(convert_ascii_to_string).astype('U').values
    df.comments = df.comments.apply(str).astype('U').values
    df.comments = df.comments.apply(remove_URL).astype('U').values
    df.comments = df.comments.apply(remove_punct).astype('U').values
    df.comments = df.comments.apply(remove_multi_periods).astype('U').values
    df.comments = df.comments.apply(remove_multi_xmarks).astype('U').values
    df.comments = df.comments.apply(remove_multi_qmarks).astype('U').values
    df.comments = df.comments.apply(remove_non_ascii).astype('U').values
    df.comments = df.comments.apply(get_sentences, k = keyword).astype('U').values
    df.comments = df.comments.apply(remove_whitespace).astype('U').values
    df.comments = df.comments.apply(replace_period).astype('U').values
    df.replace('', np.nan, inplace=True)
    df.dropna(inplace=True)
    df.drop_duplicates(inplace=True)
    for old, new in replace_words.items():
        df.comments = df.comments.str.replace(old, new, regex=False)
    #print(df.comments.to_markdown())
    with open(os.getcwd() + r'/data/sentence_samples/%s.txt'%keyword, 'a') as f:
        np.savetxt(f,df.comments, fmt='%s', newline='')
    #with open(os.getcwd() + r'/data/sentence_samples/%s.txt'%keyword,'w+', encoding = 'utf-8') as f:
        #print(*df.comments.astype('U').values, file = f )#sep = "\n", file=f)
    
    
    #np.savetxt(os.getcwd() + r'/data/sentence_samples/%s.txt'%keyword, df.comments, fmt='%s', newline='')


In [39]:
for i in keyword_list:
         saveResults(getData(i),i)